# Tutorial 01: Zeeman energy term

> Interactive online tutorial:
> [![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/ubermag/oommfc/master?filepath=docs%2Fipynb%2Findex.ipynb)

Zeeman energy density is computed as

$$w_\text{z} = -\mu_{0}M_\text{s}\mathbf{m}\cdot\mathbf{H}$$

where $\mu_{0}$ is the magnetic constant, $M_\text{s}$ is the magnetisation saturation, $\mathbf{m}$ is the normalised ($|\mathbf{m}|=1$) magnetisation, and $\mathbf{H}$ is the external magnetic field. Zeeman energy term tends to align all magnetic moments parallel to the external magnetic field.

In `oommfc`, $M_\text{s}$ and $\mathbf{m}$ are part of the magnetisation field `system.m`. Therefore, only external magnetic field $\mathbf{H}$ should be provided as an input parameter to uniquely define the Zeeman energy term. $\mathbf{H}$ can be constant in space or spatially varying.

## Spatially constant $\mathbf{H}$

Let us start by assembling a simple simple simulation where $\mathbf{H}$ does not vary in space. The sample is a "one-dimensional" chain of magnetic moments.

In [1]:
import oommfc as oc

p1 = (-10e-9, 0, 0)
p2 = (10e-9, 1e-9, 1e-9)
cell = (1e-9, 1e-9, 1e-9)
mesh = oc.Mesh(p1=p1, p2=p2, cell=cell)

The system has a Hamiltonian, which consists of only Zeeman energy term.

In [2]:
H = (0, 0, 1e6)  # external magnetic field (A/m)
system = oc.System(name='zeeman-constant-H')
system.hamiltonian = oc.Zeeman(H=H)

We are going to minimise the system's energy using `oommfc.MinDriver` later. Therefore, we do not have to define the system's dynamics equation. Finally, we need to define the system's magnetisation (`system.m`). We are going to make it random with $M_\text{s}=8\times10^{5} \,\text{Am}^{-1}$

In [3]:
import random
import discretisedfield as df

Ms = 8e5  # saturation magnetisation (A/m)

def m_fun(pos):
    return [2*random.random()-1 for i in range(3)]

system.m = df.Field(mesh, dim=3, value=m_fun, norm=Ms)

The magnetisation, we set is

In [4]:
system.m.k3d_vectors(color_field=system.m.z)

Output()

Now, we can minimise the system's energy by using `oommfc.MinDriver`.

In [5]:
md = oc.MinDriver()
md.drive(system)

2019/11/12 14:25: Running OOMMF (zeeman-constant-H.mif) ... (0.9 s)


We expect that now all magnetic moments are aligned parallel to the external magnetic field (in the $z$-direction).

In [6]:
system.m.k3d_vectors(color_field=system.m.z)

Output()

Finally, we can delete the files created by `oommfc`.

In [7]:
system.delete()

## Spatially varying $\mathbf{H}$

There are two different ways how a parameter can be made spatially varying, by using:
1. Dictionary
2. `discretisedfield.Field`

### Dictionary

In order to define a parameter using a dictionary, regions must be defined in the mesh. Regions are defined as a dictionary, whose keys are the strings and values are `discretisedfield.Region` objects, which take two corner points of the region as input parameters. 

In [8]:
p1 = (-10e-9, 0, 0)
p2 = (10e-9, 1e-9, 1e-9)
cell = (1e-9, 1e-9, 1e-9)
regions = {'region1': df.Region(p1=(-10e-9, 0, 0), p2=(0, 1e-9, 1e-9)),
           'region2': df.Region(p1=(0, 0, 0), p2=(10e-9, 1e-9, 1e-9))}
mesh = oc.Mesh(p1=p1, p2=p2, cell=cell, regions=regions)

Let us say we want to apply the external magnetic field $\mathbf{H}$ in region 1 in the $x$-direction and in region 2 in the negative $z$-direction. `H` is now defined as a dictionary:

In [9]:
H = {'region1': (1e6, 0, 0), 'region2': (0, 0, -1e6)}

The system object is

In [10]:
system = oc.System(name='zeeman-dict-H')
system.hamiltonian = oc.Zeeman(H=H)
system.m = df.Field(mesh, dim=3, value=m_fun, norm=Ms)

Its magnetisation is

In [11]:
system.m.k3d_vectors(color_field=system.m.z)

Output()

After we minimise the energy

In [12]:
md.drive(system)

2019/11/12 14:25: Running OOMMF (zeeman-dict-H.mif) ... (1.0 s)


The magnetisation is as we expected.

In [13]:
system.m.k3d_vectors(color_field=system.m.z)

Output()

In [14]:
system.delete()

### `discretisedfield.Field`

Let us say that the magnetisation field varies in space as

$$\mathbf{H}(x, y, z) = (c^{2}x, 0, c)$$

where $c=10^{9}$ and the entire field is normalised with $H = 10^{6} \,\text{Am}^{-1}$. The value of a spatially varying field is set using a Python function.

In [15]:
def H_fun(pos):
    x, y, z = pos
    c = 1e9
    return (c*c*x, 0, c)

The external magnetic field is

In [16]:
H = df.Field(mesh, dim=3, value=H_fun, norm=1e6)

The system is

In [17]:
system = oc.System(name='zeeman-field-H')
system.hamiltonian = oc.Zeeman(H=H)
system.m = df.Field(mesh, dim=3, value=m_fun, norm=Ms)

and its magnetisation is

In [18]:
system.m.k3d_vectors(color_field=system.m.z)

Output()

After the energy minimisation, the magnetisation is:

In [19]:
md.drive(system)
system.m.k3d_vectors(color_field=system.m.z)

2019/11/12 14:25: Running OOMMF (zeeman-field-H.mif) ... (1.1 s)


Output()

In [20]:
system.delete()

## Other

More details on various functionality can be found in the [API Reference](https://oommfc.readthedocs.io/en/latest/).